# Caderno de testes e análises

## Criação de uma função de interpolação de uma curva de juros a partir dos preços dos títulos

In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
from scipy.optimize import *

MONTHS_PGTO_COUPON = [2, 5, 8, 11]

In [2]:
# Inicia a data base
# dtToday = datetime.today()
dtToday = datetime.strptime("2018-06-15", "%Y-%m-%d")
VNA = 3085.363738 # 15/06/2018

In [3]:
# formato das informacoes dos bonds
# preco: preço na data base do bond
# dtVcto: data de vencimento do bond
# freqCoupon: quantidade de pagamentos de coupom num ano
# bonds = [{'preco':3235.23, 'dtVcto': '2020-08-15', 'freqCoupon':2},{'preco':3167.5, 'dtVcto': '2024-08-15', 'freqCoupon':2},
#          {'preco':3153.56, 'dtVcto': '2026-08-15', 'freqCoupon':2}, {'preco':3116.795, 'dtVcto': '2035-05-15', 'freqCoupon':2},
#          {'preco':3119.82, 'dtVcto': '2045-05-15', 'freqCoupon':2}, {'preco':3165.815, 'dtVcto': '2050-08-15', 'freqCoupon':2}]
bonds = [{'preco':3238.99, 'dtVcto': '2020-08-15', 'freqCoupon':2},{'preco':3176.73, 'dtVcto': '2024-08-15', 'freqCoupon':2},
         {'preco':3165.08, 'dtVcto': '2026-08-15', 'freqCoupon':2}, {'preco':3135.92, 'dtVcto': '2035-05-15', 'freqCoupon':2},
         {'preco':3144.03, 'dtVcto': '2045-05-15', 'freqCoupon':2}, {'preco':3191.73, 'dtVcto': '2050-08-15', 'freqCoupon':2}]

# encontrando o maior ano de vencimento para criar a lista de prazos
dtVctoBond = []
for bond in bonds:
    dtVctoBond.append(datetime.strptime(bond['dtVcto'], '%Y-%m-%d'))

maxVcto = max(dtVctoBond)
print(maxVcto)

dayVcto = maxVcto.day
monthVcto = maxVcto.month
yearVcto = maxVcto.year

dayBase = dtToday.day
monthBase = dtToday.month
yearBase = dtToday.year


#

2050-08-15 00:00:00


In [4]:
# # criando vetor com os prazos de pagamento de Cupom
# PzPgtoCupom = []
# mtzCoupon = []
# for year in range(dtToday.year, maxVcto.year + 1, 1):
#     for month in MONTHS_PGTO_COUPON:
#         if (datetime(year, month, 15) > dtToday and datetime(year, month, 15) <= maxVcto):
#             PzPgtoCupom.append((datetime(year, month, 15) - dtToday).days)
# #             print("Data Fluxo:", (datetime(year, month, 15)))
#             vetorCoupon = []
#             for bond in bonds:
#                 bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
#                                  + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
#                                  for i in range(0, bond['freqCoupon'])]
#                 if (month in bondPmtMonths):
# #                     print("1: Bond", bond['dtVcto'])
#                     vetorCoupon.append(1)
#                 else:
# #                     print("0: Bond", bond['dtVcto'])
#                     vetorCoupon.append(0)
#             mtzCoupon.append(vetorCoupon)
# mtzCoupon = np.transpose(mtzCoupon)
# # print(mtzCoupon)
# # print(PzPgtoCupom)

In [5]:
# criando vetor com os prazos de pagamento de Cupom e MTZ AUX
PzPgtoCupom = []
mtzCoupon = []
for year in range(dtToday.year, maxVcto.year + 1, 1):
    for month in MONTHS_PGTO_COUPON:
        if (datetime(year, month, 15) > dtToday and datetime(year, month, 15) <= maxVcto):
            PzPgtoCupom.append((datetime(year, month, 15) - dtToday).days)
#             print("Data Fluxo:", (datetime(year, month, 15)))
            vetorCoupon = []
            for bond in bonds:
                bondPmtMonths = [(datetime.strptime(bond['dtVcto'], '%Y-%m-%d') 
                                 + relativedelta(months=+(int(12/bond['freqCoupon']) * i))).month 
                                 for i in range(0, bond['freqCoupon'])]
                if (month in bondPmtMonths) and datetime(year, month, 15) <= (datetime.strptime(bond['dtVcto'], '%Y-%m-%d')):
#                     print("1: Bond", bond['dtVcto'])
                    vetorCoupon.append(1)
                else:
#                     print("0: Bond", bond['dtVcto'])
                    vetorCoupon.append(0)
            mtzCoupon.append(vetorCoupon)
mtzCoupon = np.transpose(mtzCoupon)

In [6]:
# PENSAR EM COMO FAZER A MULTIPLICAÇÃO DE MATRIZ
# PENSAR EM COMO ORGANIZAR A MATRIZ PARA FAZER A MINIMIZAÇÃO
# TEM QUE CONSIDERAR O VNA TB -- LEMBRAR!


### Criando a matriz de cupoms e do preço dos bonds

In [7]:
# criando o vetor de cupons
vlCupom = ((1 + 0.06)**(0.5) -1) * VNA
arrayvlCupom = [vlCupom] * len(PzPgtoCupom)

# criando a matriz de principais
mtzPrincipal = []
for k, bond in enumerate(bonds):
    mtzPrincipal.append(VNA * (np.array(PzPgtoCupom) == (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days))
    
# Somando a matriz de cupons e de principais
mtzVlCupom = (arrayvlCupom * mtzCoupon) + mtzPrincipal
# mtzVlCupom

# Criando a matriz de preços dos bonds
b =[]
for bond in bonds:
    b.append(bond['preco'])

### Criando a função de minimização pelo método não linear

In [8]:
# def PrecoBond_OK(xGuess):
#     x = [i for i in xGuess]
#     F = np.empty((len(xGuess)))
#     F[0] = (
#         arrayvlCupom[0] / (1+x[0])**(PzPgtoCupom[0]/360)
#     + arrayvlCupom[1] / (1+x[0])**(PzPgtoCupom[1]/360)
#     + arrayvlCupom[2] / (1+x[0])**(PzPgtoCupom[2]/360)
#     + arrayvlCupom[3] / (1+x[0])**(PzPgtoCupom[3]/360)
#     + arrayvlCupom[4] / (1+x[0])**(PzPgtoCupom[4]/360)
#     - b[0]
#         )
#     return F

# def PrecoBond(xGuess):
#     x = [i for i in xGuess]
#     F = np.zeros((len(xGuess)))
#     for i in range(0, len(xGuess)):
#         for k in range(0, len((PzPgtoCupom))):
#             F[i] += (arrayvlCupom[k] / (1+x[i])**(PzPgtoCupom[k]/360))
#         F[i] += -b[i]
#     return F

# # Fazendo a função de forma Matricial
# # DCMat = matriz com os Guess das taxas, esse cara precisa ser do tamanho da quantidade de bonds.
# # !- preciso definir quais prazos estão relacionados à qual taxa (uma mesma taxa pode ser usada em mais de um ponto)
# # PzMat é o vetor com os prazos já dividido por 360
# # DFMat é a matriz com as taxas elevadas aos prazos ((1+tx) ^ (Pz360)) 
# # F é a soma dos fluxos descontados menos o preço do bond a mercado
# def PrecoBondMatriz_OK(xGuess):
#     DCMat = np.mat(1+np.mat(xGuess))
#     PzMat = np.mat(PzPgtoCupom)/360
#     DFMat = np.power(DCMat,PzMat)
#     F = np.sum(np.mat(arrayvlCupom) / DFMat) - b
#     return F

In [13]:
def PrecoBondMatrizMin(xGuess):
    xGuess = list(reversed(xGuess))
    DCMat = np.zeros(len(PzPgtoCupom))
    for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
    PzMat = np.mat(PzPgtoCupom)/360
    DFMat = np.power(DCMat,PzMat)
    F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) - np.mat(b).T
    F = np.asarray(F).reshape(-1)
#     print(F,b)
    return F
######################################################################################################
##################################### funcao sem subtrair o preço do bond ############################
######################################################################################################

def PrecoBondMatriz(xGuess):
    xGuess = list(reversed(xGuess))
    DCMat = np.zeros(len(PzPgtoCupom))
    for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
    PzMat = np.mat(PzPgtoCupom)/360
    DFMat = np.power(DCMat,PzMat)
    F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) # - np.mat(b).T
    F = np.asarray(F).reshape(-1)
#     print(F,b)
    return F


In [10]:
# xGuess = list(reversed(xGuess))
# print(xGuess)
# for k, bond in enumerate(reversed(bonds)):
#     print(k, (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days)
# np.array(PzPgtoCupom)
# DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days ]
# for k, bond in enumerate(reversed(bonds)):
#     print(np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days)
#     DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
# DCMat

NameError: name 'z3' is not defined

In [14]:
xGuess = [0.1] * len(bonds)

# print(xGuess)
# z1 = fsolve(PrecoBond,xGuess)
# z2 = fsolve(PrecoBond_OK,xGuess)
z3 = fsolve(PrecoBondMatrizMin,xGuess)
# PrecoBondMatriz(xGuess)
# print(b)
# z * 100
# print(z1, z2, z3)
print(z3)
PrecoBondMatriz(z3)

[ 0.04416953  0.0575217   0.05859974  0.05821452  0.05819694  0.05827041]


array([ 3238.98999999,  3176.73000005,  3165.07999998,  3135.92      ,
        3144.02999998,  3191.72999989])

In [18]:
# mtzVlCupom[0]

xGuess = z3
xGuess = list(reversed(xGuess))
DCMat = np.zeros(len(PzPgtoCupom))
for k, bond in enumerate(reversed(bonds)):
        DCMat[np.array(PzPgtoCupom) <= (datetime.strptime(bond['dtVcto'], "%Y-%m-%d") - dtToday).days] = (1+xGuess[k])
PzMat = np.mat(PzPgtoCupom)/360
DFMat = np.power(DCMat,PzMat)

F = np.sum(np.mat(mtzVlCupom) / DFMat, axis=1) - np.mat(b).T
F = np.asarray(F).reshape(-1)
# print(F) 

# for k, bond in enumerate(bonds):
#     print('Calculado: {0:.2f}, bond: {1:.2f}'.format((np.sum(mtzVlCupom[k] / DFMat)), b[k]))
#     print(type(b[k]))
# DFMat
DCMat
# np.mat(mtzVlCupom) / DFMat,

array([ 1.04416953,  1.04416953,  1.04416953,  1.04416953,  1.04416953,
        1.04416953,  1.04416953,  1.04416953,  1.04416953,  1.0575217 ,
        1.0575217 ,  1.0575217 ,  1.0575217 ,  1.0575217 ,  1.0575217 ,
        1.0575217 ,  1.0575217 ,  1.0575217 ,  1.0575217 ,  1.0575217 ,
        1.0575217 ,  1.0575217 ,  1.0575217 ,  1.0575217 ,  1.0575217 ,
        1.05859974,  1.05859974,  1.05859974,  1.05859974,  1.05859974,
        1.05859974,  1.05859974,  1.05859974,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05821452,  1.05821452,
        1.05821452,  1.05821452,  1.05821452,  1.05819694,  1.05

$\begin{bmatrix} A_{11} && A_{12} \\
          A_{21} && A_{22}
   \end{bmatrix}$

In [ ]:
# # tentar criar uma matriz que possua o número de linhas = número de bonds, sendo cada linha o valor de x1, x2, etc.
# # o número de colunas seria = à qtd de datas de pagamento de cupom
# mtzAux = []
# for k, bond in enumerate(bonds):
# #     print(k, bond)
#     x_linha = [k] * len(PzPgtoCupom)
#     mtzAux.append(x_linha)
    
# print(mtzAux)